In [1]:
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras import backend as K


import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2021-12-14 04:55:22.687584: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-14 04:55:22.687606: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def build_model(input_shape=(100, 100, 3), dropout=0.7, learnRate=0.01):
	model = Sequential()

	model.add(Conv2D(32,(3,3),input_shape=input_shape))
	model.add(Activation('relu'))
	model.add(MaxPooling2D(pool_size=(2,2)))

	model.add(Conv2D(16,(3,3)))
	model.add(Activation('relu'))
	model.add(MaxPooling2D(pool_size=(2,2)))

	model.add(Flatten())
	model.add(Dropout(dropout))
	model.add(Dense(8,activation='relu'))
	model.add(Dense(1,activation='sigmoid'))
	model.compile(loss='binary_crossentropy',\
				optimizer=optimizers.SGD(learning_rate=learnRate, momentum=0.9),metrics=['acc']
				)


	return model

In [3]:
train_dir = '../data/train'

train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(100, 100),
                                                    batch_size=20,
                                                    class_mode='binary'
                                                    )

Found 824 images belonging to 2 classes.


In [4]:
valid_dir = '../data/validation'

val_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = val_datagen.flow_from_directory(valid_dir,
                                                        target_size=(100, 100),
                                                        batch_size=20,
                                                        class_mode='binary')

Found 274 images belonging to 2 classes.


In [5]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn=build_model, verbose=0)

grid = dict(
	input_shape=[(100, 100, 3), (75,75,3)],
	learnRate=[1e-2, 3e-2],
	dropout=[0.7, 0.5],
	batch_size=[10, 20], 
	epochs=[10]
)


In [6]:
image_number = train_generator.n  
batch_size = 20
steps = image_number // batch_size

import array


X , y = [] , []
for i in range(steps):
    a , b = train_generator.next()
    X.extend(a)
    y.extend(b)

X = np.array(X)

X.shape

(820, 100, 100, 3)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

searcher = RandomizedSearchCV(estimator=model, n_jobs=-1, cv=3,
	param_distributions=grid, scoring="accuracy", verbose=0, random_state=100)


searchResults = searcher.fit(X,y)

bestScore = searchResults.best_score_
bestParams = searchResults.best_params_
print("Best score is {:.3f} with {}".format(bestScore,
	bestParams))


In [8]:
bestScore = searchResults.best_score_
bestParams = searchResults.best_params_
print("Best score is {:.3f} with {}".format(bestScore,
	bestParams))

Best score is 0.956 with {'learnRate': 0.01, 'input_shape': (100, 100, 3), 'epochs': 10, 'dropout': 0.5, 'batch_size': 10}
